In [6]:
import os
import yaml, openai
from langchain import PromptTemplate
from langchain.chains import LLMChain
from langchain.llms import OpenAI, OpenAIChat

# ChatGPT API Basics

In [7]:
with open('cadentials.yaml') as f:
    credentials = yaml.load(f, Loader=yaml.FullLoader)

os.environ['OPENAI_API_KEY'] = credentials['OPENAI_API_KEY']
os.environ['HUGGINGFACEHUB_API_TOKEN'] = credentials['HUGGINGFACEHUB_API_TOKEN']
openai.api_key = credentials['OPENAI_API_KEY']

In [8]:
response = openai.ChatCompletion.create(
                                model="gpt-3.5-turbo",
                                messages=[
                                        {"role": "assistant", "content": "You are a helpful assistant."},
                                        {"role": "user", "content": "Hello what kind of assistant are you?"},
                                    ]
                                )
response

<OpenAIObject chat.completion id=chatcmpl-72Gf0fYWY3kqLHrSHUEKWfJSpBj7Q at 0x246d2788db0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "I am an AI-powered language model assistant designed to help you with various tasks like answering questions, providing information, assisting with tasks, and more. How may I assist you today?",
        "role": "assistant"
      }
    }
  ],
  "created": 1680775054,
  "id": "chatcmpl-72Gf0fYWY3kqLHrSHUEKWfJSpBj7Q",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 36,
    "prompt_tokens": 27,
    "total_tokens": 63
  }
}

### Chat Mark-up Language (CML) Token System

            <|im_start|>system
            You are ChatGPT, a large language model trained by OpenAI. Answer as concisely as possible.
            Knowledge cutoff: 2021-09-01
            Current date: 2023-03-01<|im_end|>
            <|im_start|>user
            How are you<|im_end|>
            <|im_start|>assistant
            I am doing well!<|im_end|>
            <|im_start|>user
            How are you now?<|im_end|>

### system tells what kind of role the LLM should play. Key thing to note is that Token system which discribedmay changes based on the role of the LLM. 

In [9]:
messages = [
            {"role": "assistant", "content": "You are a helpful assistant."},
            {"role": "user", "content": "Hello what kind of assistant are you?"},
            ]

In [10]:
conversation_total_tokens = 0

while True:
    message = input("Human: ")
    if message=='exit':
        print(f"{conversation_total_tokens} tokens used in total in this conversation")
        break
    if message:
        messages.append(
                        {
                        "role": "user", 
                        "content": message
                        },
                        )
        response = openai.ChatCompletion.create(
                                                model="gpt-3.5-turbo", 
                                                messages=messages
                                                )
    
    reply = response.choices[0].message.content
    total_tokens = response.usage['total_tokens']
    conversation_total_tokens += total_tokens
    print(f"ChatGPT: {reply} \n {total_tokens} tokens used")
    messages.append({"role": "assistant", "content": reply})

ChatGPT: Hello Isuru! I am an AI language model created by OpenAI, so I don't have a name like a human. You can call me OpenAI if you like. How may I assist you today? 
 88 tokens used
ChatGPT: As an AI language model, I don't have the capacity to compare and judge myself against other models or products. While I can certainly help you with language-related tasks, my capabilities and limitations are determined by my training dataset, programming, and other technical factors rather than marketing claims or comparisons. Nevertheless, I'd be happy to answer any questions you have or help you in any way I can. 
 202 tokens used
ChatGPT: Yes, I can generate word embeddings using a pre-trained language model such as GPT-2 or Word2Vec. Word embeddings are numerical representations of words that are generated by analyzing large amounts of text data. They enable computers to understand the semantic relationships between words, such as synonyms, antonyms, and context. These embeddings can be use

#### I'm Impressed about GPTx

# ChatGPT with LangChain

In [12]:
prefix_messages = [
                    {
                    "role": "system", 
                    "content": "You are a helpful history professor named Kate."
                    }
                  ]

llm = OpenAIChat(
                model_name='gpt-3.5-turbo', 
                temperature=0, 
                prefix_messages=prefix_messages,
                max_tokens = 256
                )

In [13]:
template = """Take the following question: {user_input}

Answer it in an informative and intersting but conscise way for someone who is new to this topic."""

prompt = PromptTemplate(
                        template=template, 
                        input_variables=["user_input"]
                        )


#### example 01

In [16]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

user_input = "Describe Large Language Models and how they work."

llm_chain.run(user_input)

'Large Language Models are computer programs that use artificial intelligence to generate human-like language. They work by analyzing vast amounts of text data, such as books, articles, and websites, to learn the patterns and structures of language. This allows them to generate new text that is similar in style and tone to the original data.\n\nOne of the most well-known examples of a Large Language Model is GPT-3, which has been trained on a massive dataset of over 45 terabytes of text. This model can generate a wide range of text, from news articles to poetry to computer code.\n\nLarge Language Models are used in a variety of applications, such as chatbots, language translation, and content creation. They have the potential to revolutionize the way we interact with technology and communicate with each other. However, there are also concerns about the ethical implications of these models, such as the potential for bias and the impact on human creativity.'

#### example 02

In [17]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

user_input = "Describe the GPT-3 model and how it works."

llm_chain.run(user_input)

"GPT-3 is a language model that uses artificial intelligence to generate human-like text. It works by analyzing vast amounts of text data and learning patterns and relationships between words and phrases. This allows it to generate coherent and contextually appropriate responses to prompts or questions. Essentially, it's like a very advanced version of predictive text on your phone, but on a much larger scale. It has the potential to revolutionize the way we interact with technology and communicate with each other."